In [2]:
# -*- coding: utf-8 -*-
"""
전처리 전용 (통합 실행)
로직(유지):
  - 모든 줄을 '한 줄'로 병합 → 맞춤법/띄어쓰기 교정
  - '목표주가/목표가/TP(단어경계)' → 'num' 치환
  - 숫자 → 'num' 치환

부수 로직(기존 스타일 유지):
  - 매수/매도 각각 최신 LIMIT_PER_CLASS개 선택
  - 세이프가드: A) FULL → B) 교정만 → C) 병합만 → D) 기본정리만 → E) 스킵
  - 출력: preprocessed.json, preprocess_debug.tsv

입력 폴더:
  ~/Desktop/매도 , ~/Desktop/매수   (각 폴더의 .txt)
출력:
  ~/Desktop/결과_매수매도_학습/preprocessed.json

"""

import re, json
from pathlib import Path

# ===== 경로/설정 =====
HOME = Path.home()
NEG_DIR = HOME / "Desktop" / "매도"
POS_DIR = HOME / "Desktop" / "매수"
OUT_DIR = HOME / "Desktop" / "결과_매수매도_학습"
OUT_DIR.mkdir(parents=True, exist_ok=True)

LIMIT_PER_CLASS = 2350    
MIN_CHARS_AFTER  = 80

# ===== 인코딩 스마트 리더 =====
def read_text_smart(p: Path) -> str:
    encodings = ["utf-8", "utf-8-sig", "cp949", "euc-kr", "iso-8859-1"]
    for enc in encodings:
        try:
            return p.read_text(encoding=enc)
        except Exception:
            continue
    try:
        return p.read_bytes().decode("utf-8", errors="ignore")
    except Exception:
        return ""

# ===== 기본 정리(개행은 나중에 제거) =====
def basic_trim_keep_newlines(t: str) -> str:
    t = re.sub(r'[\u00A0\u200B-\u200D]+', ' ', t)       
    t = t.replace('\r\n', '\n').replace('\r', '\n')     
    t = re.sub(r'[ \t]+', ' ', t)                   
    return t.strip()

# ===== 모든 줄을 '한 줄'로 병합 =====
def merge_all_to_one_line(text: str) -> str:
    t = basic_trim_keep_newlines(text)
    t = t.replace('\n', '')                           
    t = re.sub(r'\s+', ' ', t).strip()
    return t

# ===== 맞춤법/띄어쓰기 교정 =====
def correct_text_single_line(text: str) -> str:
    txt = text
    try:
        from pykospacing import Spacing
        txt = Spacing()(txt)
    except Exception:
        pass
    try:
        from hanspell import spell_checker
        res = spell_checker.check(txt)
        if hasattr(res, "checked") and isinstance(res.checked, str):
            txt = res.checked
    except Exception:
        pass
    return txt.strip()

# ===== '목표가/TP' → 'num' 치환 (tp는 단어 경계) =====
_KOR_TP = r'(?:목\s*표\s*주\s*가|목\s*표\s*가)'
_TP_WORD = r'(?<![A-Za-z0-9])tp(?![A-Za-z0-9])'
TP_MASK_PATTERN = re.compile(
    rf'(?P<kw>{_KOR_TP}|{_TP_WORD})(?P<pp>은|는|이|가|을|를|와|과|로|으로)?',
    re.IGNORECASE
)

def _particle_fix(pp: str) -> str:
    if not pp:
        return ''
    mapping = {'는': '은', '가': '이', '를': '을', '와': '과', '로': '으로'}
    return mapping.get(pp, pp)

def mask_target_keywords_to_num(text: str) -> str:
    def _repl(m: re.Match) -> str:
        pp = m.group('pp') or ''
        return 'num' + _particle_fix(pp)
    return TP_MASK_PATTERN.sub(_repl, text)

# ===== 숫자 → 'num' 치환 =====
NUM_PATTERN_TO_NUM = re.compile(
    r"""(?x)
    (?:[+-]?\d{1,3}(?:,\d{3})+(?:\.\d+)?%?)|
    (?:[+-]?\d+\.\d+%?)|
    (?:[+-]?\d+%?)|
    (?:\d+\s*만원|\d+\s*억원|\d+\s*조원|\d+\s*원)
    """
)
def mask_numbers_to_num(text: str) -> str:
    return NUM_PATTERN_TO_NUM.sub("num", text)

# ===== 최신 날짜순 N개 선택 =====
def read_latest_txts(dirpath: Path, limit: int):
    files = [p for p in dirpath.glob("*.txt") if p.is_file()]
    def extract_date(fname: str):
        nums = re.findall(r"\d{8}", fname)
        return int(nums[-1]) if nums else 0
    files.sort(key=lambda x: extract_date(x.name), reverse=True)
    files = files[:limit]
    print(f"[선택] {dirpath.name}: {len(files)}개 (최신순)")
    texts, names = [], []
    for p in files:
        raw = read_text_smart(p)
        if raw is None:
            raw = ""
        texts.append(raw)
        names.append(p.name)
    return texts, names

# ===== 전처리(세이프가드 포함) =====
def preprocess_full(doc: str) -> str:
    merged = merge_all_to_one_line(doc)
    corrected = correct_text_single_line(merged) if merged else ""
    if not corrected:
        corrected = merged
    step_tp = mask_target_keywords_to_num(corrected)
    final_text = mask_numbers_to_num(step_tp).strip()
    return final_text

def preprocess_with_safeguard(doc: str) -> str:
    # A) FULL
    t = preprocess_full(doc)
    if len(t) >= MIN_CHARS_AFTER:
        return t
    # B) 교정만
    merged = merge_all_to_one_line(doc)
    only_correct = correct_text_single_line(merged) if merged else ""
    if len(only_correct) >= MIN_CHARS_AFTER:
        return only_correct
    # C) 병합만
    if len(merged) >= MIN_CHARS_AFTER:
        return merged
    # D) 기본정리만(개행 제거 X)
    basic = basic_trim_keep_newlines(doc)
    if len(basic) >= MIN_CHARS_AFTER:
        return basic
    # E) 그래도 짧으면 빈 문자열
    return ""

def main():
    # 0) 데이터 로드(최신 N개씩)
    neg_raw, neg_names = read_latest_txts(NEG_DIR, limit=LIMIT_PER_CLASS)
    pos_raw, pos_names = read_latest_txts(POS_DIR, limit=LIMIT_PER_CLASS)

    X_raw = neg_raw + pos_raw
    y     = [0]*len(neg_raw) + [1]*len(pos_raw)
    names = neg_names + pos_names

    # 1) 전처리 + 세이프가드
    X_clean, y_clean, kept_names, strat = [], [], [], []
    skipped = 0

    for doc, label, fname in zip(X_raw, y, names):
        t = preprocess_full(doc)
        s = "FULL"
        if len(t) < MIN_CHARS_AFTER:
            t = correct_text_single_line(merge_all_to_one_line(doc))
            s = "CORRECT_ONLY"
        if not t or len(t) < MIN_CHARS_AFTER:
            t = merge_all_to_one_line(doc)
            s = "MERGE_ONLY"
        if not t or len(t) < MIN_CHARS_AFTER:
            t = basic_trim_keep_newlines(doc)
            s = "BASIC_ONLY"

        if not t or len(t) < MIN_CHARS_AFTER:
            skipped += 1
            continue

        X_clean.append(t)
        y_clean.append(label)
        kept_names.append(fname)
        strat.append(s)

    print(f"[세이프가드] 최종 유지 {len(X_clean)}개 / 스킵 {skipped}개 (기준: {MIN_CHARS_AFTER}자)")

    # 2) 저장
    (OUT_DIR / "preprocessed.json").write_text(
        json.dumps({"texts": X_clean, "labels": y_clean}, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

    with open(OUT_DIR / "preprocess_debug.tsv", "w", encoding="utf-8") as f:
        f.write("filename\tstrategy\tlen\n")
        for txt, s, nm in zip(X_clean, strat, kept_names):
            f.write(f"{nm}\t{s}\t{len(txt)}\n")

    print(f"[저장 완료] {OUT_DIR / 'preprocessed.json'}")
    print(f"[디버그] {OUT_DIR / 'preprocess_debug.tsv'}")

if __name__ == "__main__":
    main()


[선택] 매도: 2343개 (최신순)
[선택] 매수: 2350개 (최신순)
[세이프가드] 최종 유지 4671개 / 스킵 22개 (기준: 80자)
[저장 완료] C:\Users\bsbcn\Desktop\결과_매수매도_학습\preprocessed.json
[디버그] C:\Users\bsbcn\Desktop\결과_매수매도_학습\preprocess_debug.tsv


In [3]:
import json, numpy as np
from pathlib import Path
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

# ===== 경로 =====
HOME = Path.home()
OUT_DIR = HOME / "Desktop" / "결과_매수매도_학습"
DATA_PATH = OUT_DIR / "preprocessed.json"
SEED = 42

# ===== 데이터 로드 =====
with open(DATA_PATH, encoding="utf-8") as f:
    data = json.load(f)
X, y = data["texts"], data["labels"]
print(f"[데이터] 총 {len(X)}개 문서 로드 완료")

# ===== stratified split (셔플 + 계층) =====
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=SEED, stratify=y, shuffle=True
)
print(f"[분할] train={len(X_train)}, val={len(X_val)} (stratified)")

train_ds = Dataset.from_dict({"text": X_train, "label": y_train})
val_ds   = Dataset.from_dict({"text": X_val,   "label": y_val})

# ===== 토크나이저 / 모델 =====
MODEL_NAME = "snunlp/KR-FinBERT-SC"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def tok_fn(batch):
    return tokenizer(batch["text"], truncation=True, max_length=512)

train_tok = train_ds.map(tok_fn, batched=True, remove_columns=["text"])
val_tok   = val_ds.map(tok_fn,   batched=True, remove_columns=["text"])
data_collator = DataCollatorWithPadding(tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2, ignore_mismatched_sizes=True
)

# ===== 평가 지표 =====
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
    }

args = TrainingArguments(
    output_dir=str(OUT_DIR / "checkpoints"),
    learning_rate=2e-5,                  
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=4,      
    num_train_epochs=6,                   
    warmup_ratio=0.06,                   
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    fp16=True,
    report_to="none",
    seed=SEED,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=1e-4)]
)

trainer.train()

save_dir = OUT_DIR / "final_model"
save_dir.mkdir(parents=True, exist_ok=True)
trainer.save_model(str(save_dir))
tokenizer.save_pretrained(str(save_dir))
print(f"[완료] 모델 저장: {save_dir}")



[데이터] 총 4671개 문서 로드 완료
[분할] train=4203, val=468 (stratified)


Map:   0%|          | 0/4203 [00:00<?, ? examples/s]

Map:   0%|          | 0/468 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-FinBERT-SC and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\bsbcn\anaconda3\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\bsbcn\AppData\Local\Temp\ipykernel_18520\1200860915.py:86: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
0,0.550300,0.385275,0.839744,0.839532
1,0.372600,0.363955,0.850427,0.850096
2,0.335000,0.401113,0.846154,0.845022
4,0.221100,0.414853,0.837607,0.837417
5,0.207300,0.427412,0.835470,0.835301


[완료] 모델 저장: C:\Users\bsbcn\Desktop\결과_매수매도_학습\final_model


In [2]:
# -*- coding: utf-8 -*-
"""
KR-FinBERT (snunlp/KR-FinBERT-SC) 미세조정 + 에폭0 배치 로깅
A) 콘솔 출력 (epoch 0 배치별)
B) CSV 저장  (epoch 0 배치별): ~/Desktop/결과_매수매도_학습/step_logs_epoch0.csv
"""
import json, csv
import numpy as np
import torch
from pathlib import Path
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

# ===== 경로/설정 =====
HOME = Path.home()
OUT_DIR = HOME / "Desktop" / "결과_매수매도_학습"
DATA_PATH = OUT_DIR / "preprocessed.json"
SEED = 42

CSV_PATH = OUT_DIR / "step_logs_epoch0.csv"  # (B)

# ===== 데이터 로드 =====
with open(DATA_PATH, encoding="utf-8") as f:
    data = json.load(f)
X, y = data["texts"], data["labels"]
print(f"[데이터] 총 {len(X)}개 문서 로드 완료")

# ===== stratified split =====
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=SEED, stratify=y, shuffle=True
)
print(f"[분할] train={len(X_train)}, val={len(X_val)} (stratified)")

train_ds = Dataset.from_dict({"text": X_train, "label": y_train})
val_ds   = Dataset.from_dict({"text": X_val,   "label": y_val})

# ===== 토크나이저 / 모델 =====
MODEL_NAME = "snunlp/KR-FinBERT-SC"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def tok_fn(batch):
    return tokenizer(batch["text"], truncation=True, max_length=512)

train_tok = train_ds.map(tok_fn, batched=True, remove_columns=["text"])
val_tok   = val_ds.map(tok_fn,   batched=True, remove_columns=["text"])
data_collator = DataCollatorWithPadding(tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2, ignore_mismatched_sizes=True
)

# ===== 평가 지표 =====
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1_macro": f1_score(labels, preds, average="macro"),
    }

# ===== CSV 준비 (B) =====
def init_csv(path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", newline="", encoding="utf-8") as f:
        wr = csv.writer(f)
        wr.writerow(["global_step", "epoch", "loss_step", "acc_step", "lr"])

init_csv(CSV_PATH)

# ===== 커스텀 트레이너: compute_loss에서 콘솔+CSV 로깅 (epoch 0만) =====
class LoggingTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._buf_correct = 0
        self._buf_loss_sum = 0.0
        self._buf_count = 0
        self._buf_nsamples = 0

    def _flush_if_needed(self):
        # 버퍼가 grad_accum_steps만큼 쌓였으면 스텝 요약을 1번만 출력
        if self._buf_count == self.args.gradient_accumulation_steps:
            acc_step  = self._buf_correct / self._buf_nsamples if self._buf_nsamples else float("nan")
            loss_step = self._buf_loss_sum / self._buf_nsamples if self._buf_nsamples else float("nan")
            lr = (self.optimizer.param_groups[0]["lr"]
                  if getattr(self, "optimizer", None) else -1.0)
            # 주의: global_step은 optimizer.step() 후 증가하므로, 여기서는 "현재 step 진행 중" 의미
            print(f"[epoch0 step={self.state.global_step}] loss={loss_step:.4f} accuracy={acc_step:.4f}")
            # CSV도 한 줄만 기록하고 싶다면 여기에 write 추가
            # self._write_csv([self.state.global_step, float(self.state.epoch), float(loss_step), float(acc_step), float(lr)])

            # 버퍼 초기화
            self._buf_correct = 0
            self._buf_loss_sum = 0.0
            self._buf_count = 0
            self._buf_nsamples = 0

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs = model(**inputs)
        loss = outputs.loss

        # epoch 0에서만 수집/출력
        cur_epoch = self.state.epoch
        if cur_epoch is not None and int(cur_epoch) == 0 and "labels" in inputs and hasattr(outputs, "logits"):
            with torch.no_grad():
                logits = outputs.logits
                labels = inputs["labels"]
                preds = logits.argmax(dim=-1)
                n = labels.size(0)
                correct = (preds == labels).sum().item()

                # 손실이 'mean'일 때 샘플수 가중 합으로 누적
                self._buf_correct += correct
                self._buf_loss_sum += float(loss.detach().cpu()) * n
                self._buf_nsamples += n
                self._buf_count += 1

                self._flush_if_needed()

        return (loss, outputs) if return_outputs else loss
'''

class LoggingTrainer(Trainer):
    def _write_csv(self, row):
        with open(CSV_PATH, "a", newline="", encoding="utf-8") as f:
            wr = csv.writer(f)
            wr.writerow(row)

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        outputs = model(**inputs)
        loss = outputs.loss

        try:
            # 배치 지표 계산
            if "labels" in inputs and hasattr(outputs, "logits"):
                with torch.no_grad():
                    preds = outputs.logits.argmax(dim=-1)
                    acc = (preds == inputs["labels"]).float().mean().item()

                # 현재 epoch/step
                cur_epoch = self.state.epoch  # float 또는 None
                gstep = self.state.global_step

                # LR (가드)
                lr = None
                if getattr(self, "optimizer", None):
                    lr = self.optimizer.param_groups[0]["lr"]
                else:
                    try:
                        lr = self._get_learning_rate()
                    except Exception:
                        lr = -1.0

                # === 핵심: 에폭 0 내부에서만 상세 로깅 ===
                if cur_epoch is not None and int(cur_epoch) == 0:
                    # A) 콘솔 출력
                    print(f"[epoch0 step={gstep}] loss={float(loss):.4f} "
                          f"acc={acc:.4f} lr={float(lr):.6g}")

                    # B) CSV 저장
                    self._write_csv([gstep, float(cur_epoch), float(loss.detach().cpu()), float(acc), float(lr)])

                    # (선택) Trainer 내부 로그에도 남기고 싶으면 주석 해제
                    # self.log({"train/loss_step": float(loss.detach().cpu()),
                    #           "train/acc_step": float(acc), "lr": float(lr)})

        except Exception:
            pass

        return (loss, outputs) if return_outputs else loss
        '''

# ===== 학습 설정 =====
args = TrainingArguments(
    output_dir=str(OUT_DIR / "checkpoints"),
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=4,        # eff. batch = 64
    num_train_epochs=6,                   # 4~6 추천
    warmup_ratio=0.06,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=1,                      # epoch 0 배치별 콘솔 로그 위해 1
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    fp16=True,
    report_to="none",
    seed=SEED,
)

trainer = LoggingTrainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=1e-4)]
)

# ===== 에폭 0 '시작 전' 사전 검증 =====
print("\n[pre-train @epoch0] 학습 시작 전 검증 평가 실행...")
pre = trainer.evaluate(metric_key_prefix="pretrain")
print(f"[pre-train @epoch0] {pre}")

# ===== 학습 =====
trainer.train()

# ===== 저장 =====
save_dir = OUT_DIR / "final_model"
save_dir.mkdir(parents=True, exist_ok=True)
trainer.save_model(str(save_dir))
tokenizer.save_pretrained(str(save_dir))
print(f"[완료] 모델 저장: {save_dir}")
print(f"[CSV 저장 완료] {CSV_PATH}")


[데이터] 총 4671개 문서 로드 완료
[분할] train=4203, val=468 (stratified)


Map:   0%|          | 0/4203 [00:00<?, ? examples/s]

Map:   0%|          | 0/468 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-FinBERT-SC and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\bsbcn\anaconda3\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\bsbcn\AppData\Local\Temp\ipykernel_23924\2923973766.py:78: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `LoggingTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)



[pre-train @epoch0] 학습 시작 전 검증 평가 실행...


early stopping required metric_for_best_model, but did not find eval_f1_macro so early stopping is disabled


[pre-train @epoch0] {'pretrain_loss': 0.7454280853271484, 'pretrain_model_preparation_time': 0.001, 'pretrain_accuracy': 0.4807692307692308, 'pretrain_f1_macro': 0.3941021337808679, 'pretrain_runtime': 2.2936, 'pretrain_samples_per_second': 204.05, 'pretrain_steps_per_second': 6.54}
[epoch0 step=0] loss=0.7153 accuracy=0.5000


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1 Macro
0,0.391500,0.396625,0.001000,0.831197,0.830714


[epoch0 step=1] loss=0.7097 accuracy=0.5156
[epoch0 step=2] loss=0.7345 accuracy=0.4531
[epoch0 step=3] loss=0.7147 accuracy=0.5000
[epoch0 step=4] loss=0.7583 accuracy=0.3594
[epoch0 step=5] loss=0.7348 accuracy=0.4219
[epoch0 step=6] loss=0.7096 accuracy=0.4688
[epoch0 step=7] loss=0.7073 accuracy=0.4375
[epoch0 step=8] loss=0.6648 accuracy=0.6094
[epoch0 step=9] loss=0.6893 accuracy=0.5312
[epoch0 step=10] loss=0.6633 accuracy=0.6406
[epoch0 step=11] loss=0.6776 accuracy=0.5625
[epoch0 step=12] loss=0.6634 accuracy=0.5938
[epoch0 step=13] loss=0.6507 accuracy=0.6875
[epoch0 step=14] loss=0.5845 accuracy=0.7500
[epoch0 step=15] loss=0.6163 accuracy=0.6719
[epoch0 step=16] loss=0.6029 accuracy=0.7188
[epoch0 step=17] loss=0.5755 accuracy=0.6719
[epoch0 step=18] loss=0.6837 accuracy=0.5625
[epoch0 step=19] loss=0.5400 accuracy=0.7031
[epoch0 step=20] loss=0.5921 accuracy=0.6719
[epoch0 step=21] loss=0.5472 accuracy=0.7500
[epoch0 step=22] loss=0.5335 accuracy=0.7344
[epoch0 step=23] lo

KeyboardInterrupt: 